# Marktstammdatenregister Solarpanels in *.geojson

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import math

### ein Kreis pro Eintrag (überlappende Kreise)

In [2]:
# CSV-Datei laden (mit deutschem Dezimal- und Spaltentrennzeichen)
df = pd.read_csv("StromerzeugerSolarKoordinaten.csv", sep=';')

# Spaltennamen automatisch erkennen
lat_col = [col for col in df.columns if 'breit' in col.lower()][0]
lon_col = [col for col in df.columns if 'läng' in col.lower() or 'lon' in col.lower()][0]
kwp_col = [col for col in df.columns if 'installierte leistung' in col.lower()][0]

# Kommazahlen in Punktnotation umwandeln und in float konvertieren
df[lat_col] = df[lat_col].astype(str).str.replace(',', '.').astype(float)
df[lon_col] = df[lon_col].astype(str).str.replace(',', '.').astype(float)
df[kwp_col] = df[kwp_col].astype(str).str.replace(',', '.').astype(float)

# Fläche berechnen (1 kWp ≈ 6 m²)
df["Flaeche_m2"] = df[kwp_col] * 6

# Punkt-Geometrien erzeugen
geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
gdf_points = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# In metrisches Koordinatensystem (Web Mercator) transformieren
gdf_projected = gdf_points.to_crs(epsg=3857)

# Kreise mit Fläche erzeugen (Radius = √(Fläche / π))
gdf_projected["geometry"] = [
    Point(pt.x, pt.y).buffer(math.sqrt(area / math.pi))
    for pt, area in zip(gdf_projected.geometry, gdf_projected["Flaeche_m2"])
]

# Zurück in WGS84 für GeoJSON-Export
gdf_final = gdf_projected.to_crs(epsg=4326)

# Export als GeoJSON
gdf_final.to_file("solaranlagen_flaechen.geojson", driver="GeoJSON")


### gruppierte Einträge, wenn mehrere Einträge an einem Standort -> keine überlappenden Kreise

In [4]:
# CSV-Datei laden (deutsches Format: Semikolon als Trennzeichen, Komma als Dezimaltrennzeichen)
df = pd.read_csv("StromerzeugerSolarKoordinaten.csv", sep=';')

# Relevante Spalten identifizieren
lat_col = [col for col in df.columns if 'breit' in col.lower()][0]
lon_col = [col for col in df.columns if 'läng' in col.lower() or 'lon' in col.lower()][0]
kwp_col = [col for col in df.columns if 'installierte leistung' in col.lower()][0]

# Kommazahlen in Punktnotation umwandeln und in float konvertieren
df[lat_col] = df[lat_col].astype(str).str.replace(',', '.').astype(float)
df[lon_col] = df[lon_col].astype(str).str.replace(',', '.').astype(float)
df[kwp_col] = df[kwp_col].astype(str).str.replace(',', '.').astype(float)

# Gruppieren nach Koordinaten und installierte Leistung summieren
grouped = df.groupby([lat_col, lon_col], as_index=False)[kwp_col].sum()

# Fläche berechnen (1 kWp ≈ 6 m²)
grouped["Flaeche_m2"] = grouped[kwp_col] * 6

# Punkt-Geometrien erzeugen
geometry = [Point(xy) for xy in zip(grouped[lon_col], grouped[lat_col])]
gdf_points = gpd.GeoDataFrame(grouped, geometry=geometry, crs="EPSG:4326")

# In metrisches Koordinatensystem transformieren
gdf_projected = gdf_points.to_crs(epsg=3857)

# Kreise mit Fläche erzeugen (Radius = √(Fläche / π))
gdf_projected["geometry"] = [
    Point(pt.x, pt.y).buffer(math.sqrt(area / math.pi))
    for pt, area in zip(gdf_projected.geometry, gdf_projected["Flaeche_m2"])
]

# Zurück in WGS84 für GeoJSON-Export
gdf_final = gdf_projected.to_crs(epsg=4326)

# Export als GeoJSON
gdf_final.to_file("solaranlagen_flaechen_gruppiert.geojson", driver="GeoJSON")
